In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import joblib

# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data = pd.read_csv('0727_masking_labeling_data.csv')

data.columns

Index(['Unnamed: 0', 'number', 'title', 'title_morphed',
       'title_essential_morphed', 'title_removeNN_essential_morphed',
       'title_eomi_giho_essential_morphed',
       'title_eomi_giho_essential_removeNN_morphed', 'title_morphed_masked',
       'title_essential_morphed_masked',
       'title_removeNN_essential_morphed_masked',
       'title_eomi_giho_essential_morphed_masked',
       'title_eomi_giho_essential_removeNN_morphed_masked', 'name', 'date',
       'answer_state', 'contents', 'text_morphed', 'text_essential_morphed',
       'text_removeNN_essential_morphed', 'text_eomi_giho_essential_morphed',
       'text_eomi_giho_essential_removeNN_morphed', 'text_morphed_masked',
       'text_essential_morphed_masked',
       'text_removeNN_essential_morphed_masked',
       'text_eomi_giho_essential_morphed_masked',
       'text_eomi_giho_essential_removeNN_morphed_masked', 'has_attachment',
       'answer_contents', 'registration_number', 'charge_name', 'charge',
       'answer

In [3]:
titles = data.columns[3:13]
texts = data.columns[17:27]

In [4]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
import joblib
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.style.use('default')

font_location = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname = font_location).get_name()
matplotlib.rc('font',family = font_name)

model_scores_save_path = 'C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/scores/ensemble/'
visualization_save_path = 'C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/visualizations/'

# Initialize the dictionary with 'scores' and 'cases_best_scores' keys
All_model_scores = {}
ensemble_models = {}
best_models = {}

target_columns = ['aggr', '욕설_모욕', '비꼼_시비', '반복', '요지불명', '저격성 민원']
aggressive_columns = ['aggr', '욕설_모욕', '비꼼_시비', '요지불명']
target_column_type = {'aggr': 'aggressive', '욕설_모욕': 'aggressive', '비꼼_시비': 'aggressive', '반복': 'neutral', '요지불명': 'aggressive', '저격성 민원': 'targeted'}

for target_column in target_columns:

    print(f'start_{target_column}_training...\n')
    All_model_scores[target_column] = {'scores': {}, 'cases_best_scores': {}}

    for title, text in zip(titles, texts):
        selected_columns = [title, text, 'aggr', '욕설_모욕', '비꼼_시비', '반복', '요지불명', '저격성 민원']
        data1 = data[selected_columns][data[target_column] == 0]
        data2 = data[selected_columns][data[target_column] == 1]
        data1 = data1[300:300 + len(data2)]
        
        data3 = pd.concat([data1, data2])
        data_contents = data3[title] + ' ' + data3[text]
        data_labeling = data3[target_column]

        X_train, X_test, y_train, y_test = train_test_split(data_contents, data_labeling, stratify=data_labeling, test_size=0.2, random_state=42)

        pipelines = {
            'extra_trees': Pipeline([
                ('vect', TfidfVectorizer()),
                ('et', ExtraTreesClassifier())
            ]),
            'logistic_regression': Pipeline([
                ('vect', TfidfVectorizer()),
                ('lr', LogisticRegression())
            ]),
            'naive_bayes': Pipeline([
                ('vect', TfidfVectorizer()),
                ('nb', MultinomialNB())
            ]),
            'svm': Pipeline([
                ('vect', TfidfVectorizer()),
                ('svm', SVC(probability=True))
            ]),
            'hard_model': Pipeline([
                ('vect', TfidfVectorizer()),
                ('hard_model', VotingClassifier([('LR', LogisticRegression()), ('svm', SVC()), ('et', ExtraTreesClassifier()), ('nb', MultinomialNB())], voting='hard'))
            ]),
            'soft_model': Pipeline([
                ('vect', TfidfVectorizer()),
                ('soft_model', VotingClassifier([('LR', LogisticRegression()), ('svm', SVC(probability=True)), ('et', ExtraTreesClassifier()), ('nb', MultinomialNB())], voting='soft'))
            ])
        }

        case_model_scores = {}
        best_model_name = None
        best_test_score = 0
        best_cv_score = 0
        best_confusion_matrix = None
        best_classification_report = None
        best_fp_rate = float('inf')
        best_fn_rate = float('inf')

        # 모델 훈련 및 평가
        for model_name, pipeline in pipelines.items():
            print(f"Training {model_name} for {target_column}...")

            X_train_tfidf = pipeline.named_steps['vect'].fit_transform(X_train)
            X_test_tfidf = pipeline.named_steps['vect'].transform(X_test)

            print(f"{model_name} - X_train shape: {X_train_tfidf.shape}")
            print(f"{model_name} - X_test shape: {X_test_tfidf.shape}")

            pipeline.fit(X_train, y_train)

            cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

            train_score = pipeline.score(X_train, y_train)
            test_score = pipeline.score(X_test, y_test)
            print(f"{model_name} Train set score: {train_score:.4f}")
            print(f"{model_name} Test set score: {test_score:.4f}")
            print(f"{model_name} Cross-validation scores: {cv_scores}")
            print(f"{model_name} Cross-validation mean score: {cv_scores.mean():.4f}\n")

            if test_score >= 0.8:
                y_pred = pipeline.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                cr = classification_report(y_test, y_pred, output_dict=True)
                fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])  # 위양성 비율
                fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])  # 위음성 비율

                case_model_scores[model_name] = {
                    'test_score': test_score,
                    'cv_scores': cv_scores.tolist(),  # Convert numpy array to list for JSON serialization
                    'cv_mean_score': cv_scores.mean(),
                    'confusion_matrix': cm.tolist(),  # Convert numpy array to list for JSON serialization
                    'classification_report': cr,
                    'fp_rate': fp_rate,
                    'fn_rate': fn_rate
                }

                # 최고 점수 갱신
                if test_score > best_test_score:
                    best_test_score = test_score
                    best_cv_score = cv_scores.mean()
                    best_model_name = model_name
                    best_confusion_matrix = cm
                    best_classification_report = cr
                    best_fp_rate = fp_rate
                    best_fn_rate = fn_rate

            model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/ensemble', f'{target_column}_{text}_{model_name}_tfidf_model.pkl')
            joblib.dump(pipeline, model_save_path)

        print(f'{text}_{target_column}_case_complete\n')

        # 각 케이스별 모델 점수 저장
        All_model_scores[target_column]['scores'][text] = case_model_scores
        # 최고 점수를 가진 모델 정보 저장
        All_model_scores[target_column]['cases_best_scores'][text] = {
            'model_name': best_model_name,
            'test_score': best_test_score,
            'cv_mean_score': best_cv_score,
            'confusion_matrix': best_confusion_matrix.tolist() if best_confusion_matrix is not None else None,  # Convert numpy array to list for JSON serialization
            'classification_report': best_classification_report,
            'fp_rate': best_fp_rate,
            'fn_rate': best_fn_rate
        }

    # 앙상블 모델 구축 및 저장
    ensemble_pipelines = [
        ('LR', LogisticRegression()),
        ('svm', SVC(probability=True)),
        ('et', ExtraTreesClassifier()),
        ('nb', MultinomialNB())
    ]

    if target_column == '저격성 민원':
        weights = [2 if name == 'LR' else 1 for name, _ in ensemble_pipelines]
    elif target_column in aggressive_columns:
        weights = [2 if name == 'et' else 1 for name, _ in ensemble_pipelines]
    else:
        weights = [1] * len(ensemble_pipelines)  # 중립의 경우 모든 모델에 동일 가중치

    hard_ensemble_model = VotingClassifier(ensemble_pipelines, voting='hard')
    soft_ensemble_model = VotingClassifier(ensemble_pipelines, voting='soft', weights=weights)

    for ensemble_type, ensemble_model in [('hard', hard_ensemble_model), ('soft', soft_ensemble_model)]:
        ensemble_pipeline = Pipeline([
            ('vect', TfidfVectorizer()),
            ('ensemble', ensemble_model)
        ])

        ensemble_pipeline.fit(X_train, y_train)
        ensemble_test_score = ensemble_pipeline.score(X_test, y_test)
        y_pred_ensemble = ensemble_pipeline.predict(X_test)
        cm_ensemble = confusion_matrix(y_test, y_pred_ensemble)
        cr_ensemble = classification_report(y_test, y_pred_ensemble, output_dict=True)
        fp_rate_ensemble = cm_ensemble[0][1] / (cm_ensemble[0][1] + cm_ensemble[0][0])
        fn_rate_ensemble = cm_ensemble[1][0] / (cm_ensemble[1][0] + cm_ensemble[1][1])

        ensemble_models[f'{target_column}_{ensemble_type}'] = {
            'test_score': ensemble_test_score,
            'confusion_matrix': cm_ensemble.tolist(),
            'classification_report': cr_ensemble,
            'fp_rate': fp_rate_ensemble,
            'fn_rate': fn_rate_ensemble
        }

        # Save ensemble model
        ensemble_model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/ensemble', f'{target_column}_{ensemble_type}_ensemble_model.pkl')
        joblib.dump(ensemble_pipeline, ensemble_model_save_path)

        # Visualize confusion matrix
        plt.figure(figsize=(10, 7))
        sns.heatmap(cm_ensemble, annot=True, fmt='g', cmap='Blues')
        plt.title(f'Confusion Matrix for {target_column} - {ensemble_type}')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig(os.path.join(visualization_save_path, f'{target_column}_{ensemble_type}_confusion_matrix.png'))
        plt.close()

    # 앙상블 결과를 반영한 베스트 모델 선택
    if target_column in aggressive_columns:
        best_fp_model_name = max(ensemble_models, key=lambda x: ensemble_models[x]['fp_rate'])
        best_models[target_column] = {
            'best_model_name': best_fp_model_name,
            'best_test_score': ensemble_models[best_fp_model_name]['test_score'],
            'best_cv_mean_score': best_cv_score,
            'best_confusion_matrix': ensemble_models[best_fp_model_name]['confusion_matrix'],
            'best_classification_report': ensemble_models[best_fp_model_name]['classification_report']
        }
    elif target_column == '저격성 민원':
        best_fn_model_name = min(ensemble_models, key=lambda x: ensemble_models[x]['fn_rate'])
        best_models[target_column] = {
            'best_model_name': best_fn_model_name,
            'best_test_score': ensemble_models[best_fn_model_name]['test_score'],
            'best_cv_mean_score': best_cv_score,
            'best_confusion_matrix': ensemble_models[best_fn_model_name]['confusion_matrix'],
            'best_classification_report': ensemble_models[best_fn_model_name]['classification_report']
        }
    else:
        best_models[target_column] = {
            'best_model_name': best_model_name,
            'best_test_score': best_test_score,
            'best_cv_mean_score': best_cv_score,
            'best_confusion_matrix': best_confusion_matrix.tolist() if best_confusion_matrix is not None else None,
            'best_classification_report': best_classification_report
        }

# Save all model scores
all_model_scores_save_path = os.path.join(model_scores_save_path, 'all_model_scores.json')
with open(all_model_scores_save_path, 'w', encoding='utf-8') as json_file:
    json.dump(All_model_scores, json_file, ensure_ascii=False, indent=4)

# Save best models
best_models_save_path = os.path.join(model_scores_save_path, 'best_models.json')
with open(best_models_save_path, 'w', encoding='utf-8') as json_file:
    json.dump(best_models, json_file, ensure_ascii=False, indent=4)

# Save ensemble models
ensemble_models_save_path = os.path.join(model_scores_save_path, 'ensemble_models.json')
with open(ensemble_models_save_path, 'w', encoding='utf-8') as json_file:
    json.dump(ensemble_models, json_file, ensure_ascii=False, indent=4)

print("All models have been trained, evaluated, and saved.")


start_aggr_training...

Training extra_trees for aggr...
extra_trees - X_train shape: (819, 11590)
extra_trees - X_test shape: (205, 11590)
extra_trees Train set score: 0.9988
extra_trees Test set score: 0.8537
extra_trees Cross-validation scores: [0.8902439  0.84756098 0.87195122 0.87195122 0.82822086]
extra_trees Cross-validation mean score: 0.8620

Training logistic_regression for aggr...
logistic_regression - X_train shape: (819, 11590)
logistic_regression - X_test shape: (205, 11590)
logistic_regression Train set score: 0.9731
logistic_regression Test set score: 0.8927
logistic_regression Cross-validation scores: [0.86585366 0.85365854 0.86585366 0.8902439  0.8404908 ]
logistic_regression Cross-validation mean score: 0.8632

Training naive_bayes for aggr...
naive_bayes - X_train shape: (819, 11590)
naive_bayes - X_test shape: (205, 11590)
naive_bayes Train set score: 0.9634
naive_bayes Test set score: 0.8000
naive_bayes Cross-validation scores: [0.82317073 0.77439024 0.76219512 0.